In [0]:
# Adapted from
# https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [153]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [154]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.9.0-rc0


### Step 1: Generate sample equations

In [0]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [0]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [195]:
#@title Model Parameters
# Parameters for the model and dataset.
global DIGIT_LEN
global ITERATIONS
TRAINING_SIZE = 50000 #@param {type:"slider", min:100, max:100000, step:100}
ITERATIONS = 20 #@param {type:"slider", min:1, max:40, step:1}
ARITH_OPERATION = "ADD" #@param ["ADD", "SUB", "MUL", "DIV"]
OPS = {"ADD":"+", "SUB":"-", "MUL":"*", "DIV":"/"}
OP = OPS[ARITH_OPERATION]
DIGITS = 3
# REVERSE = True
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.

chars = '0123456789' + OP + ' '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    if OP == "/" and b == 0:
      continue
    q = ('{}' + OP + '{}').format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(int(eval(str(a) + OP + str(b))))
    # Answers has a maximum size regarding selected OP
    # => ADD: (DIGITS + 1)
    # => SUB: (DIGITS + 1)
    # => MUL: (DIGITS * 2)
    # => DIV: (DIGITS)
    DIGIT_LENS = {"ADD":DIGITS + 1, "SUB":DIGITS + 1, "MUL":DIGITS * 2, "DIV":DIGITS}
    DIGIT_LEN = DIGIT_LENS[ARITH_OPERATION]
    ans += ' ' * (DIGIT_LEN - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 3500


In [196]:
questions[0]

'11+8   '

In [197]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGIT_LEN, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGIT_LEN)


Vectorization...


In [198]:
len(x[0])

7

In [199]:
len(questions[0])

7

In [200]:
questions[0]

'11+8   '

In [201]:
x[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [202]:
y[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [203]:
expected[0]

'19  '

In [0]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

### Step 2: Training/Validation Split

In [205]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(3150, 7, 12)
(3150, 4, 12)
Validation Data:
(350, 7, 12)
(350, 4, 12)


### Step 3: Create Model

In [206]:
# input shape: 7 digits, each being 0-9, + or space (12 possibilities)
MAXLEN, len(chars)

(7, 12)

In [207]:
from keras.models import Sequential
from keras import layers

# Try replacing LSTM, GRU, or SimpleRNN.
# RNN = layers.LSTM
RNN = layers.SimpleRNN
# RNN = layers.GRU
HIDDEN_SIZE = 128
BATCH_SIZE = 128

print('Build model...')
model = Sequential()
# encoder: simply encode the same input 4 times 
model.add(RNN(units=HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGIT_LEN))

# decoder: have 4 temporal outputs one for each of the digits of the results
# return_sequences=True tells it to keep all 4 temporal outputs, not only the final one (we need all four digits for the results)
model.add(RNN(units=HIDDEN_SIZE, return_sequences=True))

model.add(layers.Dense(name='classifier', units=len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_11 (SimpleRNN)    (None, 128)               18048     
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
simple_rnn_12 (SimpleRNN)    (None, 4, 128)            32896     
_________________________________________________________________
classifier (Dense)           (None, 4, 12)             1548      
Total params: 52,492
Trainable params: 52,492
Non-trainable params: 0
_________________________________________________________________


In [208]:
model.predict(np.array([x_val[0]]))

array([[[0.09764637, 0.05744016, 0.099774  , 0.07831941, 0.13655199,
         0.10065944, 0.05244418, 0.06259977, 0.10453914, 0.07669203,
         0.05815143, 0.07518211],
        [0.13795015, 0.06050756, 0.12652498, 0.08863117, 0.08303354,
         0.13887276, 0.06308323, 0.05320753, 0.07861404, 0.05918092,
         0.05284164, 0.05755246],
        [0.08684085, 0.0340958 , 0.19257885, 0.06994168, 0.11951664,
         0.07161497, 0.06008438, 0.08120284, 0.11759423, 0.0602358 ,
         0.05671361, 0.04958034],
        [0.07341596, 0.05103836, 0.13154328, 0.10415559, 0.1161858 ,
         0.10111824, 0.07881986, 0.06050596, 0.08307662, 0.05652578,
         0.06765418, 0.07596039]]], dtype=float32)

In [209]:
model.predict_classes(np.array([x_val[0]]))

array([[4, 5, 2, 2]])

### Step 4: Train

In [210]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, ITERATIONS + 1):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 3150 samples, validate on 350 samples
Epoch 1/1
3150/3150 [==============================] - 1s 470us/step - loss: 1.9159 - acc: 0.3521 - val_loss: 1.6905 - val_acc: 0.4079
Q 8+280   T 288  ☒ 13  
Q 52+357  T 409  ☒ 16  
Q 81+354  T 435  ☒ 13  
Q 726+265 T 991  ☒ 132 
Q 58+1    T 59   ☒ 16  
Q 23+9    T 32   ☒ 12  
Q 35+531  T 566  ☒ 16  
Q 0+108   T 108  ☒ 18  
Q 8+186   T 194  ☒ 189 
Q 23+0    T 23   ☒ 13  

--------------------------------------------------
Iteration 2
Train on 3150 samples, validate on 350 samples
Epoch 1/1
3150/3150 [==============================] - 1s 208us/step - loss: 1.6341 - acc: 0.4121 - val_loss: 1.6068 - val_acc: 0.4164
Q 83+5    T 88   ☒ 13  
Q 234+7   T 241  ☒ 122 
Q 166+5   T 171  ☒ 112 
Q 913+24  T 937  ☒ 133 
Q 718+128 T 846  ☒ 113 
Q 198+21  T 219  ☒ 109 
Q 664+91  T 755  ☒ 156 
Q 71+12   T 83   ☒ 12  
Q 945+75  T 1020 ☒ 111 
Q 5+846   T 851  ☒ 555 


In [211]:
x_val[0]

array([[False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [212]:
model.predict(np.array([x_val[0]]))

array([[[3.7009276e-03, 3.5810671e-03, 1.0574440e-02, 4.3739578e-01,
         4.8807405e-02, 4.1483968e-02, 2.9817447e-02, 3.7922636e-02,
         4.0968977e-02, 8.8375963e-02, 1.0326844e-01, 1.5410294e-01],
        [2.9645316e-04, 8.5815811e-04, 1.4335823e-01, 1.8409276e-01,
         6.3905440e-02, 9.5398515e-02, 6.7955352e-02, 5.9114885e-02,
         7.2574146e-02, 7.8789487e-02, 9.5523208e-02, 1.3813344e-01],
        [1.3186631e-03, 6.0471648e-04, 1.2651525e-01, 1.7123899e-01,
         9.0019390e-02, 7.9532973e-02, 4.9561948e-02, 7.3138244e-02,
         9.0735301e-02, 6.5991558e-02, 1.0909474e-01, 1.4224823e-01],
        [8.2240385e-01, 6.1778480e-04, 1.6634148e-02, 2.6918884e-02,
         8.2325209e-03, 1.3987025e-02, 1.5023758e-02, 1.8875485e-02,
         1.3475046e-02, 2.2834323e-02, 2.2686079e-02, 1.8311102e-02]]],
      dtype=float32)

In [213]:
model.predict_classes(np.array([x_val[0]]))

array([[3, 3, 3, 0]])